<a href="https://colab.research.google.com/github/ben900926/Intro_to_ML_final_project/blob/main/109550146_Final_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modules

In [ ]:
# download dependancy
!pip install optuna
!pip install category_encoders

# download train, test data from my drive
!gdown "1GDmfpyUQJSR30OQGop1OfEPEos9sndrl" # train.csv https://drive.google.com/file/d/1GDmfpyUQJSR30OQGop1OfEPEos9sndrl/view?usp=sharing
!gdown "1JBxfTEXZCGWfmKFQWUlEcwlBYVnIRcUh" # test.csv https://drive.google.com/file/d/1JBxfTEXZCGWfmKFQWUlEcwlBYVnIRcUh/view?usp=sharing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.5/348.5 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.4/210.4 KB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 KB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.1/147.1 KB 18.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.6/112.6 KB 13.3 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=011c543f2e9e79248c12b2763a78b86dd2e53ea8093ae45c8c26ddba1dc604e0
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip
  A

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os
import tensorflow as tf
import random
import numpy as np
import itertools
import time

# model
from sklearn.linear_model import LinearRegression, HuberRegressor, LogisticRegression
from sklearn.impute import KNNImputer
from sklearn.preprocessing import RobustScaler, StandardScaler

from sklearn.metrics import roc_auc_score
from sklearn.base import clone
# parameter search
import optuna
from optuna.pruners import PercentilePruner

# Pipeline Constructors
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import FunctionTransformer
from category_encoders import WOEEncoder

# save model
import pickle

In [ ]:
# hyperparameters
SEED = 153
NUM_TRIALS = 100

################# You have to specify the path to store model weight! ##########
MODEL_WEIGHT_PATH = "drive/MyDrive/ml_final/model/final_model.pkl"

# enable parameter search or not (if enable, result might varies depending on which parameters optuna finds)
PARA_SEARCH = False

In [ ]:
# set seed
def set_seeds(seed):
  os.environ['PYTHONHASHSEED'] = str(seed)
  random.seed(seed)
  tf.random.set_seed(seed)
  np.random.seed(seed)

# training data

In [ ]:
train_df = pd.read_csv('train.csv')
target = train_df.pop('failure')
test_df = pd.read_csv('test.csv')

# show content
with pd.option_context("display.max_rows", 10):
  display(train_df)

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
0,0,A,80.10,material_7,material_8,9,5,7,8,4,...,20.155,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100
1,1,A,84.89,material_7,material_8,9,5,14,3,3,...,17.889,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057
2,2,A,82.43,material_7,material_8,9,5,12,1,5,...,18.288,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376
3,3,A,101.07,material_7,material_8,9,5,13,2,6,...,19.060,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282
4,4,A,188.06,material_7,material_8,9,5,9,2,8,...,18.093,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26565,26565,E,158.95,material_7,material_6,6,9,6,16,4,...,19.354,NaN,12.177,17.942,10.112,15.795,18.572,16.144,NaN,729.131
26566,26566,E,146.02,material_7,material_6,6,9,10,12,8,...,19.563,11.242,14.179,20.564,10.234,14.450,14.322,13.146,16.471,853.924
26567,26567,E,115.62,material_7,material_6,6,9,1,10,1,...,19.279,11.407,16.437,17.476,8.668,15.069,16.599,15.590,14.065,750.364
26568,26568,E,106.38,material_7,material_6,6,9,2,9,4,...,19.358,11.392,17.064,17.814,14.928,16.273,15.485,13.624,12.865,730.156


# Correlation with attribute 3~9 and measurement 17

In [ ]:
def preprocessing(df_train, df_test):
    data = pd.concat([df_train, df_test])
    
    # new attribute inspired by disscusion
    data['area'] = data['attribute_2'] * data['attribute_3']
    data['m3_null'] = data['measurement_3'].isnull().astype(np.int64)
    data['m5_null'] = data['measurement_5'].isnull().astype(np.int64)
    feature = [f for f in df_test.columns if f.startswith('measurement') or f =='loading']

    # measurement 17
    full_fill_dict ={}
    full_fill_dict['measurement_17'] = {
        'A': [                'measurement_4','measurement_5','measurement_6','measurement_7','measurement_8'],
        'B': ['measurement_3','measurement_4','measurement_5',                'measurement_7',                'measurement_9'],
        'C': [                                'measurement_5','measurement_6','measurement_7','measurement_8','measurement_9'],
        'D': ['measurement_3',                'measurement_5','measurement_6','measurement_7','measurement_8'],
        'E': [                'measurement_4','measurement_5','measurement_6',                'measurement_8','measurement_9'],
        'F': [                'measurement_4','measurement_5','measurement_6','measurement_7'],
        'G': [                'measurement_4','measurement_5','measurement_6',                'measurement_8','measurement_9'],
        'H': [                'measurement_4','measurement_5',                'measurement_7','measurement_8','measurement_9'],
        'I': ['measurement_3','measurement_4',                'measurement_7','measurement_8','measurement_9']
    }

    # features without measurement
    col = [col for col in df_test.columns if 'measurement' not in col]+ ['loading', 'm3_null', 'm5_null']
    a = []
    b = []

    for x in range(3,17):
      # correlation between each measurement and measurement 3~17
      corr = np.absolute(data.drop(col, axis=1).corr()[f'measurement_{x}']).sort_values(ascending=False)
      # sorted
      corr = corr.sort_values(ascending=False)
      a.append(np.round(np.sum(corr[1:4]),3))
      b.append(f'measurement_{x}')

    # making df with correlations
    corr_df = pd.DataFrame()
    corr_df['corr_sum'] = b
    corr_df['selected_col'] = a
    corr_df = corr_df.sort_values(by = 'corr_sum',ascending=False).reset_index(drop=True)

    # select top-10 measurement with highest correlation
    for i in range(10):
      measurement_col = 'measurement_' + corr_df.iloc[i,0][12:]
      fill_dict = {}
      # correlation for each production code
      for x in data.product_code.unique() : 
          #print(measurement_col)
          corr = np.absolute(data[data.product_code == x].drop(col, axis=1).corr()[measurement_col]).sort_values(ascending=False)
          # {'measurement_8': ['measurement_17', 'measurement_2', 'measurement_0', 'measurement_3']}
          measurement_col_dic = {}
          measurement_col_dic[measurement_col] = corr[1:5].index.tolist()
          fill_dict[x] = measurement_col_dic[measurement_col]
      full_fill_dict[measurement_col] =fill_dict

    feature = [f for f in data.columns if f.startswith('measurement') or f=='loading']
    nullValue_cols = [col for col in df_train.columns if df_train[col].isnull().sum()!=0]

    for code in data.product_code.unique():
      # we are using high-correlated given measurements to predict missing measurement_col
      for measurement_col in list(full_fill_dict.keys()):
        # train model with non-null
        tmp = data[data.product_code == code]
        column = full_fill_dict[measurement_col][code]
        tmp_train = tmp[column+[measurement_col]].dropna(how='any')
        measurement_null = (tmp[column].isnull().sum(axis=1) == 0) & (tmp[measurement_col].isnull())
        tmp_test = tmp[measurement_null]

        # using HugerRegressor(linear regression that is robust to outlines) to predict missing value
        model = HuberRegressor(epsilon=1.9, max_iter=500)
        model.fit(tmp_train[column], tmp_train[measurement_col])
        measure_null_only = (data.product_code==code) & (data[column].isnull().sum(axis=1)==0) & (data[measurement_col].isnull())
        data.loc[measure_null_only, measurement_col] = model.predict(tmp_test[column])

      # now using KNN imputer to impute missing values
      model1 = KNNImputer(n_neighbors=3)
      data.loc[data.product_code==code, feature] = model1.fit_transform(data.loc[data.product_code==code, feature])

    # average of measurement
    data['measurement_avg'] = data[[f'measurement_{i}' for i in range(3, 17)]].mean(axis=1)
    df_train = data.iloc[:df_train.shape[0],:]
    df_test = data.iloc[df_train.shape[0]:,:]
    features = ['loading', 'attribute_0', 'measurement_17', 'measurement_0', 'measurement_1', 'measurement_2', 'area', 'm3_null', 'm5_null', 'measurement_avg']
    
    return df_train, df_test, features

# Cross Validation

In [ ]:
# use 2 production groups for valid set
production = train_df["product_code"].unique()
# pick 2 index out of five product codes
cmb_groups = list(itertools.combinations(production, 2))

train_index = []
valid_index = []

# pick out data using these codes
for group in cmb_groups:
  group_zero_list = train_df.loc[train_df["product_code"]==group[0], :].index
  group_one_list = train_df.loc[train_df["product_code"]==group[1], :].index
  # combine two list
  list0 = list(group_zero_list)
  list1 = list(group_one_list)
  tmp_list = list0 + list1

  # total list - test set = train set
  train_set = set(list(train_df.index)) - set(tmp_list)
  train_index.append(list(train_set))
  valid_index.append(tmp_list)


# Training Model

In [ ]:
# preprocess
train = train_df.copy()
test = test_df.copy()
x_train, x_test, features = preprocessing(train, test)

In [ ]:
# pipeline (to make preprocessing faster)
preprocessing_pp = make_pipeline(
    make_column_transformer(
        (WOEEncoder(), ['attribute_0']), # turn string into binary value
        (FunctionTransformer(np.log1p), ['loading']),
        remainder = 'passthrough'
    ),
    RobustScaler()
)

In [ ]:
######################### cross validation #######################

# main train loop here
def score(input_model):
  set_seeds(seed=SEED)

  # store some results
  test_preds = np.zeros((x_test.shape[0],)) 
  train_preds = np.zeros((x_train.shape[0],))
  scores = np.zeros(len(train_index))
  total_time = 0

  start = time.time()
  for fold, train_ in enumerate(train_index):
    # training data
    x_train_ = x_train[features].iloc[train_, :].copy()
    y_train = target[train_].copy()
    valid_indices_in_df = valid_index[fold]
    x_valid_ = x_train[features].iloc[valid_indices_in_df, :].copy()
    y_valid = target[valid_indices_in_df].copy()

    # define model and train
    model = make_pipeline(
        clone(preprocessing_pp),
        clone(input_model)
    )

    model.fit(x_train_, y_train)
    # get predictions
    valid_pred = model.predict_proba(x_valid_)[:,1]
    test_preds += model.predict_proba(x_test)[:,1] / len(train_index)
    train_preds[valid_indices_in_df] = valid_pred
    scores[fold] = roc_auc_score(y_valid, valid_pred)
    end = time.time()
    total_time += (end-start)
  
  print(f'Worst: {round(np.min(scores), 6)}')
  print(f'Average:{round(np.mean(scores), 6)}')
  print(f'Process time: ', total_time)

  return np.mean(scores), test_preds

## Parameter Search

In [ ]:
# general parameters used for t
default_params = dict(            
    solver = 'liblinear', # nice solver for small dataset
    penalty = 'l1', # xgboost, l1, l2. This one suits the most
    max_iter = 500,
    random_state = SEED,
)

# keep specified percentage of prune
pruner = PercentilePruner(
    percentile = 75,
    n_startup_trials = 15,
    n_warmup_steps = 5,
    n_min_trials = 15,
)

def parameter_search(trials):
    
    # Optuna objective function
    def objective(trial):
        model_params = dict( 
            # default 1.0
            C = trial.suggest_float(
                "C", 1e-10, 100
            ),
        )
        model = LogisticRegression(
            **default_params,
            **model_params
        )
        return score(model)[0]
    
    
    optuna.logging.set_verbosity(optuna.logging.DEBUG)
    study = optuna.create_study(pruner = pruner,direction = "maximize")
    # Close to defaults
    study.enqueue_trial({'C': 1.0})
    study.optimize(objective, n_trials=trials)
    return study

# do parameter search only if specified
params = {}
if PARA_SEARCH:
  study = parameter_search(NUM_TRIALS)
  params = study.best_params
else:
  # the one I trained
  params = {'C': 0.1635393159306732}

# Save model weight

In [ ]:
model = LogisticRegression(**default_params, **params) 
# save model
pkl_filename = MODEL_WEIGHT_PATH
with open(pkl_filename, 'wb') as pklFile:
  pickle.dump(model, pklFile)